In [4]:
import os, time

ms_path = "/dev/shm/ms_quick_tiny"

def dir_size_bytes(path):
    total = 0
    for dp, dn, files in os.walk(path):
        for f in files:
            try:
                total += os.path.getsize(os.path.join(dp, f))
            except Exception:
                pass
    return total

watch = ms_path  # directory created by the write
for i in range(30):  # ~1 minute at 2s interval
    if not os.path.exists(watch):
        print("Waiting for MS directory to appear...")
    else:
        sz = dir_size_bytes(watch)
        print(f"{time.strftime('%H:%M:%S')} size {sz/1e6:.1f} MB")
    time.sleep(2)


18:25:47 size 0.0 MB
18:25:49 size 0.0 MB
18:25:51 size 0.0 MB
18:25:53 size 0.0 MB
18:25:55 size 0.0 MB
18:25:57 size 0.0 MB
18:25:59 size 0.0 MB
18:26:01 size 0.0 MB
18:26:03 size 0.0 MB
18:26:05 size 0.0 MB
18:26:07 size 0.0 MB
18:26:09 size 0.0 MB
18:26:11 size 0.0 MB
18:26:13 size 0.0 MB
18:26:15 size 0.0 MB
18:26:17 size 0.0 MB
18:26:19 size 0.0 MB
18:26:21 size 0.0 MB
18:26:23 size 0.0 MB
18:26:25 size 0.0 MB
18:26:27 size 0.0 MB
18:26:29 size 0.0 MB
18:26:31 size 0.0 MB
18:26:33 size 0.0 MB
18:26:35 size 0.0 MB
18:26:37 size 0.0 MB
18:26:39 size 0.0 MB
18:26:41 size 0.0 MB
18:26:43 size 0.0 MB
18:26:45 size 0.0 MB


In [ ]:
# Fast small-MS write (metadata-only projection; no DATA rotation)
# - Keeps only the first N_TIMES_TO_KEEP timestamps to speed up IO
# - Recomputes UVW for a single ICRS/J2000 phase center
# - Writes to /tmp; change DEST_MS if you prefer a different location

import os, sys, time, shutil
import numpy as np
import astropy.units as u
from astropy.time import Time
from pyuvdata import UVData

# ----- Config -----
SRC_ROOT = "/data/dsa110-contimg/src"
IN_DIR = "/data/incoming_test/2025-09-05T03-12-56_HDF5"
IN_FILE = os.path.join(IN_DIR, "2025-09-05T03:12:56_sb00.hdf5")  # single subband for speed
DEST_MS = "/tmp/quick_small.ms"  # change to your preferred path
N_TIMES_TO_KEEP = 2              # keep first 2 times for a quick run
WRITE_MODEL = False              # set True to write MODEL_DATA with a point source

# ----- Path bootstrap for imports in notebook -----
if SRC_ROOT not in sys.path:
    sys.path.insert(0, SRC_ROOT)

from dsa110_contimg.conversion.helpers import (
    set_antenna_positions,
    _ensure_antenna_diameters,
    get_meridian_coords,
    set_model_column,
)
from dsa110_contimg.utils.fringestopping import calc_uvw_blt

# ----- Load UVH5 (single subband for speed) -----
t0 = time.time()
uv = UVData()
uv.read(
    IN_FILE,
    file_type="uvh5",
    run_check=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_extra=False,
)
uv.uvw_array = uv.uvw_array.astype(np.float64)

# ----- Optional: select first N_TIMES_TO_KEEP times to minimize size -----
unique_times = np.unique(uv.time_array)
if unique_times.size > N_TIMES_TO_KEEP:
    uv.select(times=unique_times[:N_TIMES_TO_KEEP], run_check=False)

# ----- Antenna geometry & diameters -----
set_antenna_positions(uv)
_ensure_antenna_diameters(uv)

# ----- Single-center (ICRS/J2000) phase center; recompute per-row UVW -----
pt_dec = uv.extra_keywords.get("phase_center_dec", 0.0) * u.rad
t_mid = Time(float(np.mean(uv.time_array)), format="jd").mjd
ra_icrs, dec_icrs = get_meridian_coords(pt_dec, t_mid)

# Build a single catalog entry
uv.phase_center_catalog = {}
pc_id = uv._add_phase_center(
    cat_name="FIELD_CENTER",
    cat_type="sidereal",
    cat_lon=float(ra_icrs.to_value(u.rad)),
    cat_lat=float(dec_icrs.to_value(u.rad)),
    cat_frame="icrs",
    cat_epoch=2000.0,
)
# Per-row ID array pointing to this center
if not hasattr(uv, "phase_center_id_array") or uv.phase_center_id_array is None:
    uv.phase_center_id_array = np.zeros(uv.Nblts, dtype=int)
uv.phase_center_id_array[:] = pc_id

# Compute UVW per time block (no DATA rotation)
nbls = uv.Nbls
times = np.unique(uv.time_array)
ant_pos = np.asarray(uv.antenna_positions)  # (Nants, 3) relative to telescope loc
ant1 = np.asarray(uv.ant_1_array[:nbls], dtype=int)
ant2 = np.asarray(uv.ant_2_array[:nbls], dtype=int)
blen = ant_pos[ant2, :] - ant_pos[ant1, :]  # (Nbls, 3)

for i, tval in enumerate(times):
    row_slice = slice(i * nbls, (i + 1) * nbls)
    time_vec = np.full(nbls, float(Time(tval, format="jd").mjd), dtype=float)
    uv.uvw_array[row_slice, :] = calc_uvw_blt(
        blen,
        time_vec,
        "J2000",
        ra_icrs,
        dec_icrs,
        obs="OVRO_MMA",
    )

# Mark as projected and ensure MS-friendly metadata
uv.phase_type = "phased"
uv.phase_center_frame = "icrs"
uv.phase_center_epoch = 2000.0
# Ascending frequency and positive channel width
uv.reorder_freqs(channel_order="freq", run_check=False)

print(f"Ntimes={uv.Ntimes}, Nbls={uv.Nbls}, Nblts={uv.Nblts}")
print("UVW finite fraction:", np.isfinite(uv.uvw_array).mean())

# ----- Write MS (fast path) -----
if os.path.exists(DEST_MS):
    shutil.rmtree(DEST_MS)

uv.write_ms(
    DEST_MS,
    clobber=True,
    run_check=False,
    check_extra=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_autos=False,
    fix_autos=False,
)
print(f"Wrote MS: {DEST_MS} in {time.time() - t0:.1f}s")

# ----- Optional: write MODEL_DATA for a point source at phase center -----
if WRITE_MODEL:
    set_model_column(DEST_MS[:-3], uv, pt_dec, ra_icrs, dec_icrs, flux_Jy=10.0)
    print("MODEL_DATA written.")


2025-10-07 17:49:03 [INFO] Setting DSA-110 antenna positions
2025-10-07 17:49:04 [INFO] Loaded dynamic antenna positions for 117 antennas


Ntimes=2, Nbls=4656, Nblts=9312
UVW finite fraction: 1.0
Wrote MS: /tmp/quick_small.ms in 12.9s


In [ ]:
# Fast small-MS write (metadata-only projection; no DATA rotation)
# - Keeps only the first N_TIMES_TO_KEEP timestamps to speed up IO
# - Recomputes UVW for a single ICRS/J2000 phase center
# - Writes to /tmp; change DEST_MS if you prefer a different location

import os, sys, time, shutil
import numpy as np
import astropy.units as u
from astropy.time import Time
from pyuvdata import UVData

# ----- Config -----
SRC_ROOT = "/data/dsa110-contimg/src"
IN_DIR = "/data/incoming_test/2025-09-05T03-12-56_HDF5"
IN_FILE = os.path.join(IN_DIR, "2025-09-05T03:12:56_sb00.hdf5")  # single subband for speed
DEST_MS = "/tmp/quick_small.ms"  # change to your preferred path
N_TIMES_TO_KEEP = 2              # keep first 2 times for a quick run
WRITE_MODEL = False              # set True to write MODEL_DATA with a point source

# ----- Path bootstrap for imports in notebook -----
if SRC_ROOT not in sys.path:
    sys.path.insert(0, SRC_ROOT)

from dsa110_contimg.conversion.helpers import (
    set_antenna_positions,
    _ensure_antenna_diameters,
    get_meridian_coords,
    set_model_column,
)
from dsa110_contimg.utils.fringestopping import calc_uvw_blt

# ----- Load UVH5 (single subband for speed) -----
t0 = time.time()
uv = UVData()
uv.read(
    IN_FILE,
    file_type="uvh5",
    run_check=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_extra=False,
)
uv.uvw_array = uv.uvw_array.astype(np.float64)

# ----- Optional: select first N_TIMES_TO_KEEP times to minimize size -----
unique_times = np.unique(uv.time_array)
if unique_times.size > N_TIMES_TO_KEEP:
    uv.select(times=unique_times[:N_TIMES_TO_KEEP], run_check=False)

# ----- Antenna geometry & diameters -----
set_antenna_positions(uv)
_ensure_antenna_diameters(uv)

# ----- Single-center (ICRS/J2000) phase center; recompute per-row UVW -----
pt_dec = uv.extra_keywords.get("phase_center_dec", 0.0) * u.rad
t_mid = Time(float(np.mean(uv.time_array)), format="jd").mjd
ra_icrs, dec_icrs = get_meridian_coords(pt_dec, t_mid)

# Build a single catalog entry
uv.phase_center_catalog = {}
pc_id = uv._add_phase_center(
    cat_name="FIELD_CENTER",
    cat_type="sidereal",
    cat_lon=float(ra_icrs.to_value(u.rad)),
    cat_lat=float(dec_icrs.to_value(u.rad)),
    cat_frame="icrs",
    cat_epoch=2000.0,
)
# Per-row ID array pointing to this center
if not hasattr(uv, "phase_center_id_array") or uv.phase_center_id_array is None:
    uv.phase_center_id_array = np.zeros(uv.Nblts, dtype=int)
uv.phase_center_id_array[:] = pc_id

# Compute UVW per time block (no DATA rotation)
nbls = uv.Nbls
times = np.unique(uv.time_array)
ant_pos = np.asarray(uv.antenna_positions)  # (Nants, 3) relative to telescope loc
ant1 = np.asarray(uv.ant_1_array[:nbls], dtype=int)
ant2 = np.asarray(uv.ant_2_array[:nbls], dtype=int)
blen = ant_pos[ant2, :] - ant_pos[ant1, :]  # (Nbls, 3)

for i, tval in enumerate(times):
    row_slice = slice(i * nbls, (i + 1) * nbls)
    time_vec = np.full(nbls, float(Time(tval, format="jd").mjd), dtype=float)
    uv.uvw_array[row_slice, :] = calc_uvw_blt(
        blen,
        time_vec,
        "J2000",
        ra_icrs,
        dec_icrs,
        obs="OVRO_MMA",
    )

# Mark as projected and ensure MS-friendly metadata
uv.phase_type = "phased"
uv.phase_center_frame = "icrs"
uv.phase_center_epoch = 2000.0
# Ascending frequency and positive channel width
uv.reorder_freqs(channel_order="freq", run_check=False)

print(f"Ntimes={uv.Ntimes}, Nbls={uv.Nbls}, Nblts={uv.Nblts}")
print("UVW finite fraction:", np.isfinite(uv.uvw_array).mean())

# ----- Write MS (fast path) -----
if os.path.exists(DEST_MS):
    shutil.rmtree(DEST_MS)

uv.write_ms(
    DEST_MS,
    clobber=True,
    run_check=False,
    check_extra=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_autos=False,
    fix_autos=False,
)
print(f"Wrote MS: {DEST_MS} in {time.time() - t0:.1f}s")

# ----- Optional: write MODEL_DATA for a point source at phase center -----
if WRITE_MODEL:
    set_model_column(DEST_MS[:-3], uv, pt_dec, ra_icrs, dec_icrs, flux_Jy=10.0)
    print("MODEL_DATA written.")


2025-10-07 17:49:03 [INFO] Setting DSA-110 antenna positions
2025-10-07 17:49:04 [INFO] Loaded dynamic antenna positions for 117 antennas


Ntimes=2, Nbls=4656, Nblts=9312
UVW finite fraction: 1.0
Wrote MS: /tmp/quick_small.ms in 12.9s


In [ ]:
# Fast small-MS write (metadata-only projection; no DATA rotation)
# - Keeps only the first N_TIMES_TO_KEEP timestamps to speed up IO
# - Recomputes UVW for a single ICRS/J2000 phase center
# - Writes to /tmp; change DEST_MS if you prefer a different location

import os, sys, time, shutil
import numpy as np
import astropy.units as u
from astropy.time import Time
from pyuvdata import UVData

# ----- Config -----
SRC_ROOT = "/data/dsa110-contimg/src"
IN_DIR = "/data/incoming_test/2025-09-05T03-12-56_HDF5"
IN_FILE = os.path.join(IN_DIR, "2025-09-05T03:12:56_sb00.hdf5")  # single subband for speed
DEST_MS = "/tmp/quick_small.ms"  # change to your preferred path
N_TIMES_TO_KEEP = 2              # keep first 2 times for a quick run
WRITE_MODEL = False              # set True to write MODEL_DATA with a point source

# ----- Path bootstrap for imports in notebook -----
if SRC_ROOT not in sys.path:
    sys.path.insert(0, SRC_ROOT)

from dsa110_contimg.conversion.helpers import (
    set_antenna_positions,
    _ensure_antenna_diameters,
    get_meridian_coords,
    set_model_column,
)
from dsa110_contimg.utils.fringestopping import calc_uvw_blt

# ----- Load UVH5 (single subband for speed) -----
t0 = time.time()
uv = UVData()
uv.read(
    IN_FILE,
    file_type="uvh5",
    run_check=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_extra=False,
)
uv.uvw_array = uv.uvw_array.astype(np.float64)

# ----- Optional: select first N_TIMES_TO_KEEP times to minimize size -----
unique_times = np.unique(uv.time_array)
if unique_times.size > N_TIMES_TO_KEEP:
    uv.select(times=unique_times[:N_TIMES_TO_KEEP], run_check=False)

# ----- Antenna geometry & diameters -----
set_antenna_positions(uv)
_ensure_antenna_diameters(uv)

# ----- Single-center (ICRS/J2000) phase center; recompute per-row UVW -----
pt_dec = uv.extra_keywords.get("phase_center_dec", 0.0) * u.rad
t_mid = Time(float(np.mean(uv.time_array)), format="jd").mjd
ra_icrs, dec_icrs = get_meridian_coords(pt_dec, t_mid)

# Build a single catalog entry
uv.phase_center_catalog = {}
pc_id = uv._add_phase_center(
    cat_name="FIELD_CENTER",
    cat_type="sidereal",
    cat_lon=float(ra_icrs.to_value(u.rad)),
    cat_lat=float(dec_icrs.to_value(u.rad)),
    cat_frame="icrs",
    cat_epoch=2000.0,
)
# Per-row ID array pointing to this center
if not hasattr(uv, "phase_center_id_array") or uv.phase_center_id_array is None:
    uv.phase_center_id_array = np.zeros(uv.Nblts, dtype=int)
uv.phase_center_id_array[:] = pc_id

# Compute UVW per time block (no DATA rotation)
nbls = uv.Nbls
times = np.unique(uv.time_array)
ant_pos = np.asarray(uv.antenna_positions)  # (Nants, 3) relative to telescope loc
ant1 = np.asarray(uv.ant_1_array[:nbls], dtype=int)
ant2 = np.asarray(uv.ant_2_array[:nbls], dtype=int)
blen = ant_pos[ant2, :] - ant_pos[ant1, :]  # (Nbls, 3)

for i, tval in enumerate(times):
    row_slice = slice(i * nbls, (i + 1) * nbls)
    time_vec = np.full(nbls, float(Time(tval, format="jd").mjd), dtype=float)
    uv.uvw_array[row_slice, :] = calc_uvw_blt(
        blen,
        time_vec,
        "J2000",
        ra_icrs,
        dec_icrs,
        obs="OVRO_MMA",
    )

# Mark as projected and ensure MS-friendly metadata
uv.phase_type = "phased"
uv.phase_center_frame = "icrs"
uv.phase_center_epoch = 2000.0
# Ascending frequency and positive channel width
uv.reorder_freqs(channel_order="freq", run_check=False)

print(f"Ntimes={uv.Ntimes}, Nbls={uv.Nbls}, Nblts={uv.Nblts}")
print("UVW finite fraction:", np.isfinite(uv.uvw_array).mean())

# ----- Write MS (fast path) -----
if os.path.exists(DEST_MS):
    shutil.rmtree(DEST_MS)

uv.write_ms(
    DEST_MS,
    clobber=True,
    run_check=False,
    check_extra=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_autos=False,
    fix_autos=False,
)
print(f"Wrote MS: {DEST_MS} in {time.time() - t0:.1f}s")

# ----- Optional: write MODEL_DATA for a point source at phase center -----
if WRITE_MODEL:
    set_model_column(DEST_MS[:-3], uv, pt_dec, ra_icrs, dec_icrs, flux_Jy=10.0)
    print("MODEL_DATA written.")


2025-10-07 17:49:03 [INFO] Setting DSA-110 antenna positions
2025-10-07 17:49:04 [INFO] Loaded dynamic antenna positions for 117 antennas


Ntimes=2, Nbls=4656, Nblts=9312
UVW finite fraction: 1.0
Wrote MS: /tmp/quick_small.ms in 12.9s


In [ ]:
# Fast small-MS write (metadata-only projection; no DATA rotation)
# - Keeps only the first N_TIMES_TO_KEEP timestamps to speed up IO
# - Recomputes UVW for a single ICRS/J2000 phase center
# - Writes to /tmp; change DEST_MS if you prefer a different location

import os, sys, time, shutil
import numpy as np
import astropy.units as u
from astropy.time import Time
from pyuvdata import UVData

# ----- Config -----
SRC_ROOT = "/data/dsa110-contimg/src"
IN_DIR = "/data/incoming_test/2025-09-05T03-12-56_HDF5"
IN_FILE = os.path.join(IN_DIR, "2025-09-05T03:12:56_sb00.hdf5")  # single subband for speed
DEST_MS = "/tmp/quick_small.ms"  # change to your preferred path
N_TIMES_TO_KEEP = 2              # keep first 2 times for a quick run
WRITE_MODEL = False              # set True to write MODEL_DATA with a point source

# ----- Path bootstrap for imports in notebook -----
if SRC_ROOT not in sys.path:
    sys.path.insert(0, SRC_ROOT)

from dsa110_contimg.conversion.helpers import (
    set_antenna_positions,
    _ensure_antenna_diameters,
    get_meridian_coords,
    set_model_column,
)
from dsa110_contimg.utils.fringestopping import calc_uvw_blt

# ----- Load UVH5 (single subband for speed) -----
t0 = time.time()
uv = UVData()
uv.read(
    IN_FILE,
    file_type="uvh5",
    run_check=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_extra=False,
)
uv.uvw_array = uv.uvw_array.astype(np.float64)

# ----- Optional: select first N_TIMES_TO_KEEP times to minimize size -----
unique_times = np.unique(uv.time_array)
if unique_times.size > N_TIMES_TO_KEEP:
    uv.select(times=unique_times[:N_TIMES_TO_KEEP], run_check=False)

# ----- Antenna geometry & diameters -----
set_antenna_positions(uv)
_ensure_antenna_diameters(uv)

# ----- Single-center (ICRS/J2000) phase center; recompute per-row UVW -----
pt_dec = uv.extra_keywords.get("phase_center_dec", 0.0) * u.rad
t_mid = Time(float(np.mean(uv.time_array)), format="jd").mjd
ra_icrs, dec_icrs = get_meridian_coords(pt_dec, t_mid)

# Build a single catalog entry
uv.phase_center_catalog = {}
pc_id = uv._add_phase_center(
    cat_name="FIELD_CENTER",
    cat_type="sidereal",
    cat_lon=float(ra_icrs.to_value(u.rad)),
    cat_lat=float(dec_icrs.to_value(u.rad)),
    cat_frame="icrs",
    cat_epoch=2000.0,
)
# Per-row ID array pointing to this center
if not hasattr(uv, "phase_center_id_array") or uv.phase_center_id_array is None:
    uv.phase_center_id_array = np.zeros(uv.Nblts, dtype=int)
uv.phase_center_id_array[:] = pc_id

# Compute UVW per time block (no DATA rotation)
nbls = uv.Nbls
times = np.unique(uv.time_array)
ant_pos = np.asarray(uv.antenna_positions)  # (Nants, 3) relative to telescope loc
ant1 = np.asarray(uv.ant_1_array[:nbls], dtype=int)
ant2 = np.asarray(uv.ant_2_array[:nbls], dtype=int)
blen = ant_pos[ant2, :] - ant_pos[ant1, :]  # (Nbls, 3)

for i, tval in enumerate(times):
    row_slice = slice(i * nbls, (i + 1) * nbls)
    time_vec = np.full(nbls, float(Time(tval, format="jd").mjd), dtype=float)
    uv.uvw_array[row_slice, :] = calc_uvw_blt(
        blen,
        time_vec,
        "J2000",
        ra_icrs,
        dec_icrs,
        obs="OVRO_MMA",
    )

# Mark as projected and ensure MS-friendly metadata
uv.phase_type = "phased"
uv.phase_center_frame = "icrs"
uv.phase_center_epoch = 2000.0
# Ascending frequency and positive channel width
uv.reorder_freqs(channel_order="freq", run_check=False)

print(f"Ntimes={uv.Ntimes}, Nbls={uv.Nbls}, Nblts={uv.Nblts}")
print("UVW finite fraction:", np.isfinite(uv.uvw_array).mean())

# ----- Write MS (fast path) -----
if os.path.exists(DEST_MS):
    shutil.rmtree(DEST_MS)

uv.write_ms(
    DEST_MS,
    clobber=True,
    run_check=False,
    check_extra=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_autos=False,
    fix_autos=False,
)
print(f"Wrote MS: {DEST_MS} in {time.time() - t0:.1f}s")

# ----- Optional: write MODEL_DATA for a point source at phase center -----
if WRITE_MODEL:
    set_model_column(DEST_MS[:-3], uv, pt_dec, ra_icrs, dec_icrs, flux_Jy=10.0)
    print("MODEL_DATA written.")


2025-10-07 17:49:03 [INFO] Setting DSA-110 antenna positions
2025-10-07 17:49:04 [INFO] Loaded dynamic antenna positions for 117 antennas


Ntimes=2, Nbls=4656, Nblts=9312
UVW finite fraction: 1.0
Wrote MS: /tmp/quick_small.ms in 12.9s
